In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

import torch

# Check if CUDA is available
if torch.cuda.is_available():
    print("CUDA is available.")
else:
    print("CUDA is not available. Check if your GPU drivers are properly installed.")


CUDA is available.


In [2]:
# !pip install datasets>=2.6.1
# !pip install git+https://github.com/huggingface/transformers
# !pip install librosa
# !pip install jiwer
# !pip install evaluate>=0.30
# #!pip install gradio
# !pip install -U accelerate
# !pip install mutagen

In [3]:
from huggingface_hub import login
# load the token from txt file
with open("HF_token.txt", "r") as f:
    HF_TOKEN = f.read().strip() # strip() removes the trailing "\n" if it exists
login(token=HF_TOKEN)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/user_7542/.cache/huggingface/token
Login successful


In [4]:
import os
import librosa
import random
import numpy as np
import IPython.display as ipd
import pickle
import pandas as pd
from datasets import Dataset
from datasets import Audio
from transformers import WhisperProcessor
import mutagen.mp3
from tqdm import tqdm
import json

NEWDATA = True
ADDTOKENS = True
NIKUD = False # False to remove the nikud
JUST_TEAMIM = False
BASE_CHAR = "@"
NUSACHIM =  ["ashkenazi", "maroko", "yerushalmi", "bavly"] #["ashkenazi", "maroko", "yerushalmi", "bavly"]


FASTTEST = False
SR = 16000
RANDOM = True 

LR = 1e-5
WARMUP_STEPS = 0
EVAL_STEPS = 500    # was 1000
SAVE_STEPS = 2000
MAX_STEPS = 20000

#base model 
BASE_MODEL_NAME = "openai/whisper-medium"
# "openai/whisper-large-v2"
# "openai/whisper-tiny"
# "BenShermaister/whisper-medium-he"
# "cantillation/whisper-medium-he-teamim-ashkenazi-01"
# "cantillation/whisper-medium-he-teamim-aviv-base"

#the new model - after training 
MODEL_NAME = "./whisper-medium-he-teamim-allNusah-11-03-24"



In [5]:
processor = WhisperProcessor.from_pretrained("openai/whisper-medium", language="hebrew", task="transcribe")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
if ADDTOKENS:
    teamim = ['֑', '֒', '֓', '֔', '֕', '֖', '֗', '֘', '֙', '֚', '֛', '֜', '֝', '֞', '֟', '֠', '֡', '֢', '֣', '֤', '֥', '֦', '֧', '֨', '֩', '֪', '֫', '֬', '֭', '֮', 'ֽ']
    if JUST_TEAMIM:
        new_tokens = [BASE_CHAR + c for c in teamim] # add the base char to the teamim (e.g. א֑)
    elif NIKUD:
        new_tokens = ['֑', '֒', '֓', '֔', '֕', '֖', '֗', '֘', '֙', '֚', '֛', '֜', '֝', '֞', '֟', '֠', '֡', '֢', '֣', '֤', '֥', '֦', '֧', '֨', '֩', '֪', '֫', '֬', '֭', '֮', '֯', 'ְ', 'ֱ', 'ֲ', 'ֳ', 'ִ', 'ֵ', 'ֶ', 'ַ', 'ָ', 'ֹ', 'ֺ', 'ֻ', 'ּ', 'ֽ', '־', 'ֿ', '׀', 'ׁ', 'ׂ', '׃', 'ׄ', 'ׅ', '׆', 'ׇ']
    else:
        new_tokens = teamim

    processor.tokenizer.add_tokens(new_tokens)
    
    
        
        
    

In [7]:
def remove_nikud(text): #TODO IMPORT FROM THE FILE 
    nikud_list = ["ֱ","ֲ","ֳ","ִ","ֵ","ֶ","ַ","ָ","ׂ","ׁ","ֹ","ּ","ֻ","ְ","ׇ"]
    for nikud in nikud_list:
        text = text.replace(nikud, "")
    return text

def just_teamim(text, base_char = BASE_CHAR):#TODO IMPORT FROM THE FILE 
    teamim = ['֑', '֒', '֓', '֔', '֕', '֖', '֗', '֘', '֙', '֚', '֛', '֜', '֝', '֞', '֟', '֠', '֡', '֢', '֣', '֤', '֥', '֦', '֧', '֨', '֩', '֪', '֫', '֬', '֭', '֮', 'ֽ']
    new_text = ""
    for char in text:
        if char in teamim:
            new_text += base_char
            new_text += char
        elif char == " ":
            new_text += " "
    return new_text

In [8]:
# path = "/content/drive/Othercomputers/My Laptop/Project/data/PocketTorah/"
# path = "/content/drive/MyDrive/PocketTorah/"
path = "."

In [9]:
class parashat_hashavua_dataset:
        def __init__(self,few_data=False, train = True ,validation=False, test=False, num_of_words_in_sample = 15, random = False, prob_for_num_of_parts=[], nusachim=["ashkenazi"]):
                self.data = []
                self.few_data = few_data
                self.load_data(train, validation, test, nusachim=nusachim)
                if JUST_TEAMIM:
                        self.data['text'] = self.data['text'].apply(just_teamim)
                elif not NIKUD:
                        self.data['text'] = self.data['text'].apply(remove_nikud)
                self.data = self.data[self.data['text'] != ""] # remove empty texts (and their audio)
                self.num_of_words_in_sample = num_of_words_in_sample
                self.random = random
                self.start = 0
                self.is_eval_set = validation
                self.prob_for_num_of_parts = prob_for_num_of_parts if prob_for_num_of_parts else [1/self.num_of_words_in_sample for i in range(self.num_of_words_in_sample)]
                # prob_for_num_of_parts - the probability to take 1, 2, 3, etc. parts.
                # example of prob_for_num_of_parts: [0.1, 0.2, 0.3, 0.4] means that the probability to take 1 part is 0.1, 2 parts is 0.2, etc.

        def __getitem__(self, index):
                if self.is_eval_set:
                        audio, text_tokens, _ = self.get_sequence_(index*self.num_of_words_in_sample, num_of_words=self.num_of_words_in_sample)
                else:
                        if self.random:
                                # ensure that the sum of probabilities is 1
                                if np.sum(self.prob_for_num_of_parts) != 1:
                                        self.prob_for_num_of_parts = self.prob_for_num_of_parts / np.sum(self.prob_for_num_of_parts)
                                # get the number of parts
                                num_of_parts = np.random.choice(np.arange(1, len(self.prob_for_num_of_parts)+1), p=self.prob_for_num_of_parts)
                        # get the sequence
                                audio, text_tokens = self.get_random_words_sequence_audio_tokens(num_of_words=self.num_of_words_in_sample, num_of_parts=num_of_parts)
                        else:
                                audio, text_tokens, _ = self.get_sequence_(index, num_of_words=self.num_of_words_in_sample)

                # compute log-Mel input features from input audio array
                input_features = processor.feature_extractor(audio, sampling_rate=SR).input_features[0]
                # compute input length of audio sample in seconds
                input_length = len(audio) / SR
                # processor.tokenizer.decode(text_tokens)
                return {"input_features": input_features, "input_length": input_length, "labels": text_tokens}

        def __len__(self):
                if self.is_eval_set:
                        return int(len(self.data)/self.num_of_words_in_sample)
                else:
                        if self.random:
                                # high number because of the augmentation
                                return 100000
                        else:
                                # The length is the (number of word in the data)/(number of words in sequance)
                                return len(self.data)

        def get_sequence_audio_text(self, sequence):
                audio = np.concatenate(sequence['audio'].values)
                text = " ".join(sequence['text'])
                audio_len = len(audio) / 16000
                text_tokens = processor.tokenizer.encode(text)
                text_len = len(text_tokens)
                return sequence, audio, text, audio_len, text_tokens, text_len
        
        def load_data(self, train, validation, test, nusachim=["ashkenazi"]): # ashkenazi, maroko, yerushalmi, bavly
                if  (train==True and validation==False and test==False):
                        self.load_data_new(nusachim,train=True, validation=False, test=False)
                elif (train==False and validation==True and test==False):
                        self.load_data_new(nusachim,train=False, validation=True, test=False)
                elif (train==False and validation==False and test==True):
                        self.load_data_new(nusachim,train=False, validation=False, test=True)
                else:
                        print("Invalid input. Please provide a valid input.")

        # methods for the new data
        def is_mp3_and_legal_length(self, filename, min_length=0.2, max_length=20):
                try:
                        audio = mutagen.mp3.MP3(filename)
                        if audio.info.length < min_length or audio.info.length > max_length:
                                return False
                        else:
                                return True
                except mutagen.MutagenError:
                        return False

        
        def is_text_with_nikud(self, text):
                for char in text:
                        if char in "ְֱֲֳִֵֶַָֹֺֻּֽ֑֖֛֢֣֤֥֦֧֪֚֭֮֒֓֔֕֗֘֙֜֝֞֟֠֡֨֩֫֬֯־ֿ׀ׁׂ׃ׅׄ׆ׇ": # string of all the nikud characters ['֑', '֒', '֓', '֔', '֕', '֖', '֗', '֘', '֙', '֚', '֛', '֜', '֝', '֞', '֟', '֠', '֡', '֢', '֣', '֤', '֥', '֦', '֧', '֨', '֩', '֪', '֫', '֬', '֭', '֮', '֯', 'ְ', 'ֱ', 'ֲ', 'ֳ', 'ִ', 'ֵ', 'ֶ', 'ַ', 'ָ', 'ֹ', 'ֺ', 'ֻ', 'ּ', 'ֽ', '־', 'ֿ', '׀', 'ׁ', 'ׂ', '׃', 'ׄ', 'ׅ', '׆', 'ׇ']
                                return True
                return False

        def is_text_and_audio_pair_legal(self, text, filename):
                if not self.is_text_with_nikud(text):
                        return False
                if not self.is_mp3_and_legal_length(filename):
                        return False
                return True

        def load_data_new(self, nusachim,train, validation, test):
                # Load dataset.json
                if train:
                        file_path = 'train_data.json'
                elif validation:
                        file_path = 'validation_data.json'
                else:
                        file_path = '03_dataset.json'

                with open(file_path, 'r', encoding='utf-8') as f:
                        predataset = json.load(f)
                
                audios = []
                texts = []
                for nusach in nusachim:
                        file_path = "dataset_" + nusach + ".npy"
                        if os.path.exists(file_path) and False: # we don't want to use the saved data as 1 file right now
                                data = np.load(file_path, allow_pickle=True).item()
                                audios.extend(data['audio'])
                                texts.extend(data['text'])
                        else:
                                if self.few_data:
                                        predataset[nusach] = predataset[nusach][:500]
                                        predataset['text'] = predataset['text'][:500]
                                        
                                missing_files = []
                                for index, audio_file in enumerate(tqdm(predataset[nusach], desc=f"Loading {nusach} nusach ({nusachim.index(nusach)+1}/{len(nusachim)})")):
                                        audio_path = os.path.join(audio_file)
                                        if self.is_text_and_audio_pair_legal(predataset['text'][index], audio_path):
                                                audio, sr = librosa.load(audio_path, sr=SR)
                                                audios.append(audio)
                                                texts.append(predataset['text'][index])
                                        else:
                                                missing_files.append((audio_path, predataset['text'][index], index))
                                # Save the missing files
                                with open('missing_files' + nusach + '.json', 'w', encoding='utf-8') as f:
                                        json.dump(missing_files, f, ensure_ascii=False, indent=4)
                                print("Num of missing files in " + nusach + " nusach: ", len(missing_files))
                                # Save the data for the next time
                                data = {"audio": audios, "text": texts}
                # create the dataset
                self.data = {"audio": audios, "text": texts}
                self.data = pd.DataFrame(self.data)


        # methods for the old data
        def prepare_transcript_str_to_list(self, text:str) -> list:
                """
                this function get a string of words and return a list of the words
                """
                return text.replace(" ׀ ", "׀ ").replace(" ׀ ", "׀ ").replace("־", "־ ").replace("[1]", "").replace("\n", "").split(" ")

        def load_data_old(self, validation):
                if validation:
                        transcript_folder = path + '/text_val'
                else:
                        transcript_folder = path + '/text'
                audio_folder = path + '/audio'
                timing_folder = path + '/time'
                audios = []
                text = []
                for filename in os.listdir(transcript_folder):
                        if filename.endswith(".txt"):
                                audio_path = os.path.join(audio_folder, filename.replace('.txt', '.mp3'))
                                transcript_path = os.path.join(transcript_folder, filename)
                                timing_path = os.path.join(timing_folder, filename)

                                audio, sr = librosa.load(audio_path, sr=16000)
                                with open(transcript_path, 'r', encoding='utf-8') as f:
                                        transcript = self.prepare_transcript_str_to_list(f.read())
                                with open(timing_path, 'r', encoding='utf-8') as f:
                                        timings = [float(time) for time in f.read().split(",")]

                                for i, (word, start_time) in enumerate(zip(transcript, timings)):
                                        if i == len(transcript) - 1:
                                                end_time = len(audio) / sr
                                        else:
                                                end_time = timings[i+1]

                                        word_audio = audio[int(start_time * sr):int(end_time * sr)]
                                        audios.append(word_audio)
                                        text.append(word)
                        data_dict = {"audio": audios, "text": text}
                        self.data = pd.DataFrame(data_dict)

        def get_data(self):
                return self.data

        def get_random_word(self):
                return random.choice(self.data)

        def get_sequence(self, start, end):
                return self.data[start:end]

        # the limit of whisper model
        # audio length of 30 seconds
        # text length of 448 tokens
        # I will take 20 words and check if the audio and text are in the limit
        def get_sequence_(self, start, num_of_words=20, random_cut_long=False):
                if start + num_of_words > len(self.data):
                        end = len(self.data)
                else:
                        end = start + num_of_words
                sequence = self.get_sequence(start, end)
                sequence, audio, text, audio_len, text_tokens, text_len = self.get_sequence_audio_text(sequence)
                if audio_len < 30 and text_len < 448:
                        return audio, text_tokens, end
                else: # cut the sequence
                        print("this sequence of ", num_of_words, " words is too long!")
                        print("sequence audio length: ", audio_len)
                        print("sequence text length(in tokens): ", text_len)
                        print("text: ", text)
                        # ipd.display(ipd.Audio(audio, rate=SR))

                        if random_cut_long:
                                # divide into 2 parts and randomaly take one of them
                                if random.randint(0, 1) == 0:
                                        start = start + int(num_of_words/2)

                        if num_of_words>=2:
                                return self.get_sequence_(start, num_of_words=int(num_of_words/2), random_cut_long=random_cut_long)
                        else:
                                return self.get_sequence_(end, num_of_words=num_of_words, random_cut_long=random_cut_long)


        def get_dataset_slice_to_sequences(self, num_of_words):
                audios = []
                labels = []
                start = 0
                while start < len(self.data):
                        audio, label_feature, start = self.get_sequence_(start,num_of_words)
                        audios.append(audio)
                        labels.append(label_feature)
                dataset = {"audios": audios, "labels": labels}
                dataset = pd.DataFrame(dataset)
                return dataset
        

        def get_random_sequence_(self, length=20):
                """
                get random sequence of "length" words
                """
                start = random.randint(0, len(self.data) - length)
                return self.get_sequence_(start)

        def get_random_sequence(self, length=20):
                """
                get random sequence of "length" words
                """
                start = random.randint(0, len(self.data) - length)
                return self.get_sequence(start, start+length)

        def get_random_words_sequence_audio_tokens(self, num_of_words, num_of_parts = None):
                """
                get sequence of random words (not logical sentences)
                createed from num_of_parts short sentences
                """
                if num_of_parts == None:
                        num_of_parts = num_of_words

                if num_of_parts > num_of_words:
                        print("num_of_parts can't be bigger than num_of_words")
                        print("so num_of_parts = num_of_words = ", num_of_words)
                        num_of_parts = num_of_words

                # num of words in each part
                num_of_words_in_parts = [num_of_words // num_of_parts + (1 if i < num_of_words % num_of_parts else 0) for i in range(num_of_parts)]

                sequence = {"audio": [], "text": []}
                for num_of_words_in_part in num_of_words_in_parts:
                        part = self.get_random_sequence(num_of_words_in_part)
                        sequence["audio"].extend(part["audio"])
                        sequence["text"].extend(part["text"])
                sequence = pd.DataFrame(sequence)
                sequence, audio, text, audio_len, text_tokens, text_len = self.get_sequence_audio_text(sequence)
                if audio_len < 30 and text_len < 448:
                        return audio, text_tokens
                else:
                        print("this sequence (of ", num_of_words, " words) is too long!")
                        print("sequence audio length: ", audio_len)
                        print("sequence text length(in tokens): ", text_len)
                        print("text: ", text)
                        # ipd.display(ipd.Audio(audio, rate=SR))
                        return self.get_random_words_sequence_audio_tokens(num_of_words, num_of_parts)


        def get_dataset_slice_to_sequences_random_words(self, num_of_words, num_of_sequences=None, times = 5):
                audios = []
                labels = []
                if num_of_sequences:
                        num_of_sequences = num_of_sequences
                else:
                        num_of_sequences = int(len(self.data)*times/num_of_words)
                for i in range(num_of_sequences):
                        audio, label_feature = self.get_random_words_sequence_audio_tokens(num_of_words)
                        audios.append(audio)
                        labels.append(label_feature)
                dataset = {"audios": audios, "labels": labels}
                dataset = pd.DataFrame(dataset)
                return dataset
        
        
        # methods for checking the data
        def get_longest_audio_index(self):
                """
                returns the index of longest audio in the dataset
                """
                index = np.argmax([len(audio) for audio in self.data['audio']])
                return index
        
        def get_longest_text_index(self):
                """
                returns the index of longest text in the dataset
                """
                index = np.argmax([len(text) for text in self.data['text']])
                return index
        
        def get_shortest_audio_index(self):
                """
                returns the index of shortest audio in the dataset
                """
                index = np.argmin([len(audio) for audio in self.data['audio']])
                return index
        
        def get_shortest_text_index(self):
                """
                returns the index of shortest text in the dataset
                """
                index = np.argmin([len(text) for text in self.data['text']])
                return index
        
        def check_the_data(self):
                """
                find the longest and shortest audio and text in the dataset
                and print and play them
                """
                index = self.get_longest_audio_index()
                print("longest audio index: ", index)
                print("longest audio text: ", self.data['text'][index])
                ipd.display(ipd.Audio(self.data['audio'][index], rate=SR))

                index = self.get_shortest_audio_index()
                print("shortest audio index: ", index)
                print("shortest audio text: ", self.data['text'][index])
                ipd.display(ipd.Audio(self.data['audio'][index], rate=SR))

                index = self.get_longest_text_index()
                print("longest text index: ", index)
                print("longest text: ", self.data['text'][index])
                ipd.display(ipd.Audio(self.data['audio'][index], rate=SR))

                index = self.get_shortest_text_index()
                print("shortest text index: ", index)
                print("shortest text: ", self.data['text'][index])
                ipd.display(ipd.Audio(self.data['audio'][index], rate=SR))


        def remove_word_by_index(self, index):
                """
                delete word from the dataset by index
                """
                if index < 0 or index >= len(self.data):
                        print("Invalid index. Please provide a valid index.")
                        return
                
                self.data.drop(index, inplace=True)
                self.data.reset_index(drop=True, inplace=True)

        def print_and_play_word_by_index(self,index):
                print(self.data['text'][index])
                ipd.display(ipd.Audio(self.data['audio'][index], rate=SR))
                

In [10]:
if NEWDATA:
    train_data = parashat_hashavua_dataset(few_data=FASTTEST, train =True ,validation=False, random=RANDOM, num_of_words_in_sample=4, nusachim=NUSACHIM) # nusachim=["ashkenazi", "maroko", "yerushalmi", "bavly"])

# else:
#     train_data = parashat_hashavua_dataset(train =True ,validation=False, random=RANDOM, num_of_words_in_sample=15, prob_for_num_of_parts = [0.05, 0.05, 0.05, 0.05, 0.1, 0.15, 0.15, 0.2, 0.2])

Loading ashkenazi nusach (1/4): 100%|████| 21900/21900 [01:34<00:00, 232.80it/s]


Num of missing files in ashkenazi nusach:  119


Loading maroko nusach (2/4): 100%|███████| 21900/21900 [01:19<00:00, 273.92it/s]


Num of missing files in maroko nusach:  685


Loading yerushalmi nusach (3/4): 100%|███| 21900/21900 [01:32<00:00, 235.75it/s]


Num of missing files in yerushalmi nusach:  298


Loading bavly nusach (4/4): 100%|████████| 21900/21900 [01:29<00:00, 245.47it/s]


Num of missing files in bavly nusach:  1495


In [11]:
# play random from the train data
# train_data.print_and_play_word_by_index(random.randint(0, len(train_data.data)))


#validation of the data
# train_data.check_the_data() 
# remove the sample index:
# train_data.remove_word_by_index(32487)


In [12]:
val_data = parashat_hashavua_dataset(few_data=FASTTEST, train=False ,validation=True,  num_of_words_in_sample=4, nusachim=NUSACHIM)
# test_data = parashat_hashavua_dataset(few_data=FASTTEST, train=False ,validation=False, test=True,  num_of_words_in_sample=4, nusachim=NUSACHIM)

print(len(train_data))
print(len(val_data))

Loading ashkenazi nusach (1/4): 100%|██████| 2766/2766 [00:12<00:00, 222.19it/s]


Num of missing files in ashkenazi nusach:  8


Loading maroko nusach (2/4): 100%|█████████| 2766/2766 [00:10<00:00, 260.05it/s]


Num of missing files in maroko nusach:  62


Loading yerushalmi nusach (3/4): 100%|█████| 2766/2766 [00:11<00:00, 233.63it/s]


Num of missing files in yerushalmi nusach:  64


Loading bavly nusach (4/4): 100%|██████████| 2766/2766 [00:11<00:00, 237.79it/s]

Num of missing files in bavly nusach:  159
100000
2154


In [13]:
path = "/home/user_7542/Desktop/project/cantillation/"

In [14]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [15]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [16]:
import evaluate
import time
import cantilLocations_evaluation


# # possible metrics : "wer", "cer", "bleu", "rouge", "sacrebleu", "sari":
# # 1. `wer`: Word Error Rate.
# # 2. `cer`: Character Error Rate.
# # 3. `bleu`: Bilingual Evaluation Understudy.
# # 4. `rouge`: Recall-Oriented Understudy for Gisting Evaluation.
# # 5. `sacrebleu`: A standardized BLEU score implementation for more consistent machine translation evaluation.
# # 6. `sari`: System Agnostic Refinement Index. 

WER_CALCULATOR = evaluate.load("wer")
def compute_metrics(pred):
    eval_list = cantilLocations_evaluation.calculate_precision_recall_f1_for_string_list_with_method_list
    pred_ids = pred.predictions
    label_ids = pred.label_ids
    
    # method to calculate the metrics(method can be "Exact", "Letter_Shift", "Word_Level", "Word_Shift")
    methods = ["Exact", "Letter_Shift", "Word_Level", "Word_Shift"]

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id
    
    start_time = time.time()
    
    # we do not want to group tokens when computing the metrics
    pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)
    
    decode_time = time.time() - start_time
    
    # evaluate the metrics
    results = eval_list(pred_str, label_str, methods)
    
    
    
    # compute the average of each metric
    avg = {}
    for i in range(4):
        avg["avg_precision_" + methods[i]] = np.mean(results[i][0])
        avg["avg_recall_" + methods[i]] = np.mean(results[i][1])
        avg["avg_f1_" + methods[i]] = np.mean(results[i][2])
    
    precision_list_exact = results[methods.index("Exact")][0]
    recall_list_exact = results[methods.index("Exact")][1]
    f1_list_exact = results[methods.index("Exact")][2]
    
    # compute the median
    precision_median_exact = np.median(precision_list_exact)
    recall_median_exact = np.median(recall_list_exact)
    f1_median_exact = np.median(f1_list_exact)
    
    
    # max and min:
    precision_max_exact = np.max(precision_list_exact)
    recall_max_exact = np.max(recall_list_exact)
    f1_max_exact = np.max(f1_list_exact)
    best_index = np.argmax(f1_list_exact)
    
    f1_min = [0, 0, 0, 0]
    recall_min = [0, 0, 0, 0]
    precision_min = [0, 0, 0, 0]
    
    for i in range(4):
        precision_min[i] = np.min(results[i][0])
        recall_min[i] = np.min(results[i][1])
        f1_min[i] = np.min(results[i][2])
    
    worst_index = [np.argmin(results[i][2]) for i in range(4)] 
    
    
    
    start_time = time.time()
    # WER
    wer = 100 * WER_CALCULATOR.compute(predictions=pred_str, references=label_str)
    
    wer_time = time.time() - start_time
    
    best_pred = pred_str[best_index]
    best_label = label_str[best_index]
    worst_pred = [pred_str[worst_index[i]] for i in range(4)]
    worst_label = [label_str[worst_index[i]] for i in range(4)]
    
    # print
    # best:
    print(f"best f1 for {methods[0]}: {f1_max_exact}\nbest pred: {best_pred}\nbest label: {best_label}\n")
    
    # worst (the worst for each method):
    for i in range(4):
        print(f"worst f1 for {methods[i]}: {f1_min[i]}\nworst pred: {worst_pred[i]}\nworst label: {worst_label[i]}\n")
    
    
    
    print("Time taken for each part:")
    print(f"Decode calculation: {decode_time} seconds")
    print(f"WER calculation: {wer_time} seconds")
    
    # matric_dict = {"wer": wer, "precision": precision_avg, "recall": recall_avg, "f1": f1_avg, "precision_median": precision_median, "recall_median": recall_median, "f1_median": f1_median, "precision_max": precision_max, "recall_max": recall_max, "f1_max": f1_max, "precision_min": precision_min, "recall_min": recall_min, "f1_min": f1_min}
    
    # create the matric_dict with the metrics
    matric_dict = {"wer": wer}
    for i in range(4):
        matric_dict["avg_precision_" + methods[i]] = avg["avg_precision_" + methods[i]]
        matric_dict["avg_recall_" + methods[i]] = avg["avg_recall_" + methods[i]]
        matric_dict["avg_f1_" + methods[i]] = avg["avg_f1_" + methods[i]]
    matric_dict["precision_median_exact"] = precision_median_exact
    matric_dict["recall_median_exact"] = recall_median_exact
    matric_dict["f1_median_exact"] = f1_median_exact
    matric_dict["precision_max_exact"] = precision_max_exact
    matric_dict["recall_max_exact"] = recall_max_exact
    matric_dict["f1_max_exact"] = f1_max_exact
    for i in range(4):
        matric_dict["precision_min_" + methods[i]] = precision_min[i]
        matric_dict["recall_min_" + methods[i]] = recall_min[i]
        matric_dict["f1_min_" + methods[i]] = f1_min[i]
    # print(matric_dict)
    return matric_dict

2024-03-07 12:14:13.177780: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-07 12:14:13.270124: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-03-07 12:14:13.270138: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-03-07 12:14:13.870636: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-

In [17]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained(BASE_MODEL_NAME, use_cache=False) # we can add "force_download=True" to download the model again

model.generation_config.language = "he"

# # initialize the last layer of the model:
# model.proj_out.__init__(model.proj_out.in_features, len(processor.tokenizer))

# # add dropout
# model.config.attention_dropout = 0.1
# model.config.dropout = 0.1
# model.config.activation_dropout = 0.1


if ADDTOKENS:
    model.resize_token_embeddings(len(processor.tokenizer))

model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

model.config.decoder_input_ids = None



In [18]:
from transformers import Seq2SeqTrainingArguments

if FASTTEST:
    batch = 1
else:
    batch = 8
training_args = Seq2SeqTrainingArguments(
    output_dir= MODEL_NAME,  # change to a repo name of your choice
    per_device_train_batch_size=batch,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=LR, # was 1e-5
    warmup_steps=WARMUP_STEPS, # was 500
    max_steps=MAX_STEPS, # was 4000
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={'use_reentrant':True}, # I added that because UserWarning: "The default value of use_reentrant will be updated to be False in the future."
    fp16=torch.cuda.is_available(), # I added that because fp16 can't be use on CPU but on cuda
    evaluation_strategy="steps",
    per_device_eval_batch_size=32,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=SAVE_STEPS,  # was 1000
    eval_steps=EVAL_STEPS,    # was 1000
    logging_steps=50, # was 25
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model= "avg_f1_Exact",# "avg_f1_..." like "avg_f1_Exact"
    greater_is_better=True, # if we use f1 score in eval so greater is better
    push_to_hub=True,
    # I added the dataloader_prefetch_factor to support newer versions of torch (now it must be int and not None. and the default is 2).
    dataloader_prefetch_factor=2, # I added that
    dataloader_num_workers=1, # I added that
    weight_decay=0.0001, # I added that
)


In [19]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

In [20]:
processor.save_pretrained(training_args.output_dir)

[]

In [21]:
def flags_warnings():
    if FASTTEST:
        for i in range(10):
            print("!!!TEST-MODE!!! \t \t to test the code only")

    if not ADDTOKENS:
        print("!!!ADDTOKENS==False!!!")

    if not NEWDATA:
        print("!!!NEWDATA==False!!!")

In [22]:
flags_warnings()

trainer_state = trainer.train()

Step,Training Loss,Validation Loss,Wer,Avg Precision Exact,Avg Recall Exact,Avg F1 Exact,Avg Precision Letter Shift,Avg Recall Letter Shift,Avg F1 Letter Shift,Avg Precision Word Level,Avg Recall Word Level,Avg F1 Word Level,Avg Precision Word Shift,Avg Recall Word Shift,Avg F1 Word Shift,Precision Median Exact,Recall Median Exact,F1 Median Exact,Precision Max Exact,Recall Max Exact,F1 Max Exact,Precision Min Exact,Recall Min Exact,F1 Min Exact,Precision Min Letter Shift,Recall Min Letter Shift,F1 Min Letter Shift,Precision Min Word Level,Recall Min Word Level,F1 Min Word Level,Precision Min Word Shift,Recall Min Word Shift,F1 Min Word Shift
500,0.196000,0.236620,33.456740,0.714840,0.724119,0.718719,0.749838,0.759855,0.754046,0.755966,0.766301,0.760329,0.864234,0.878558,0.870384,0.785714,0.800000,0.800000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.055556,0.057143
1000,0.131200,0.178529,24.501129,0.775289,0.781257,0.777665,0.802555,0.808828,0.805063,0.808248,0.815724,0.811329,0.907838,0.923498,0.914806,0.866667,0.866667,0.866667,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1500,0.083900,0.148422,20.125134,0.821894,0.827591,0.824196,0.847813,0.853680,0.850180,0.852992,0.858890,0.855379,0.938203,0.948793,0.942794,0.909091,0.916667,0.909091,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.055556,0.057143
2000,0.092900,0.140988,18.562808,0.834685,0.836902,0.835327,0.861356,0.863891,0.862144,0.865274,0.868422,0.866363,0.945702,0.952963,0.948741,0.923077,0.923077,0.916667,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.055556,0.057143
2500,0.069500,0.131788,17.233720,0.864548,0.859760,0.861692,0.891193,0.886199,0.888216,0.894685,0.889955,0.891834,0.959731,0.957346,0.958024,0.933333,0.928571,0.933333,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.052632,0.055556
3000,0.048600,0.136748,16.589538,0.860215,0.866706,0.863001,0.884050,0.890837,0.886970,0.887297,0.894476,0.890401,0.955343,0.965202,0.959735,0.933333,0.933333,0.933333,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.055556,0.057143
3500,0.055700,0.130808,16.297064,0.864459,0.868698,0.866165,0.887481,0.891968,0.889298,0.890837,0.895624,0.892791,0.957175,0.963064,0.959636,0.933333,0.933333,0.933333,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.055556,0.057143
4000,0.042100,0.119501,14.727333,0.881012,0.880044,0.880136,0.903321,0.902358,0.902436,0.906705,0.906019,0.905954,0.965204,0.966020,0.965167,0.937500,0.937500,0.942810,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.055556,0.057143
4500,0.040000,0.125513,14.645885,0.879486,0.882090,0.880364,0.901785,0.904496,0.902705,0.904909,0.907891,0.905959,0.963424,0.967595,0.965039,0.937500,0.937500,0.941176,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.055556,0.057143
5000,0.031000,0.123400,14.727333,0.870295,0.868915,0.869212,0.894940,0.893436,0.893782,0.897428,0.896525,0.896568,0.966996,0.969789,0.967957,0.937500,0.937500,0.941176,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.055556,0.057143


this sequence of  5  words is too long!
sequence audio length:  33.0188125
sequence text length(in tokens):  65
text:  וידב֥ר יהו֖ה אל־מש֥ה לאמֽר׃ והנ֛פש וכרמך֖ ל֥א תזמֽר׃ כ֣י אח֤י אב֨יה֙ ה֔וא ויה֗י
this sequence of  5  words is too long!
sequence audio length:  30.418125
sequence text length(in tokens):  101
text:  ל֥א תפא֖ר אֽחר֑יך וא֨לה֙ בנ֣י רעוא֔ל למ֤ה הרע֨ת֙ לעבד֔ך ש֣ם מ֤ת אֽהרן֙ ויקב֣ר ש֔ם בכל־לבֽבך֥ ובכל־נפשך֖ ובכל־מאדֽך׃
this sequence of  5  words is too long!
sequence audio length:  30.604
sequence text length(in tokens):  89
text:  וכל־יתד֥ת הֽחצ֖ר נחֽשת׃ וידב֥ר יהו֖ה אל־מש֥ה לאמֽר׃ בכל־לבֽבך֥ ובכל־נפשך֖ למ֥ען חיֽיך׃ ויש֗בו ובאת֡
this sequence of  5  words is too long!
sequence audio length:  31.068375
sequence text length(in tokens):  101
text:  והב֤יא את֨נו֙ אל־הא֣רץ הז֔את והקרבת֥ם אש֖ה לֽיהוֽה׃ ורח֥ץ במ֖ים וטמ֥א עד־העֽרב׃ ולֽא־תש֥א את֖ה לבדֽך׃ יהוש֤ע בן־נון֙ נ֔ער
this sequence of  5  words is too long!
sequence audio length:  30.4646875
sequence text length(in tokens):  1

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [], 'begin_suppress_tokens': [220, 50257]}


this sequence of  5  words is too long!
sequence audio length:  33.0188125
sequence text length(in tokens):  65
text:  וידב֥ר יהו֖ה אל־מש֥ה לאמֽר׃ והנ֛פש וכרמך֖ ל֥א תזמֽר׃ כ֣י אח֤י אב֨יה֙ ה֔וא ויה֗י
this sequence of  5  words is too long!
sequence audio length:  30.418125
sequence text length(in tokens):  101
text:  ל֥א תפא֖ר אֽחר֑יך וא֨לה֙ בנ֣י רעוא֔ל למ֤ה הרע֨ת֙ לעבד֔ך ש֣ם מ֤ת אֽהרן֙ ויקב֣ר ש֔ם בכל־לבֽבך֥ ובכל־נפשך֖ ובכל־מאדֽך׃
this sequence of  5  words is too long!
sequence audio length:  30.604
sequence text length(in tokens):  89
text:  וכל־יתד֥ת הֽחצ֖ר נחֽשת׃ וידב֥ר יהו֖ה אל־מש֥ה לאמֽר׃ בכל־לבֽבך֥ ובכל־נפשך֖ למ֥ען חיֽיך׃ ויש֗בו ובאת֡
this sequence of  5  words is too long!
sequence audio length:  31.068375
sequence text length(in tokens):  101
text:  והב֤יא את֨נו֙ אל־הא֣רץ הז֔את והקרבת֥ם אש֖ה לֽיהוֽה׃ ורח֥ץ במ֖ים וטמ֥א עד־העֽרב׃ ולֽא־תש֥א את֖ה לבדֽך׃ יהוש֤ע בן־נון֙ נ֔ער
this sequence of  5  words is too long!
sequence audio length:  30.4646875
sequence text length(in tokens):  1

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [], 'begin_suppress_tokens': [220, 50257]}


...

this sequence of  5  words is too long!
sequence audio length:  33.0188125
sequence text length(in tokens):  65
text:  וידב֥ר יהו֖ה אל־מש֥ה לאמֽר׃ והנ֛פש וכרמך֖ ל֥א תזמֽר׃ כ֣י אח֤י אב֨יה֙ ה֔וא ויה֗י
this sequence of  5  words is too long!
sequence audio length:  30.418125
sequence text length(in tokens):  101
text:  ל֥א תפא֖ר אֽחר֑יך וא֨לה֙ בנ֣י רעוא֔ל למ֤ה הרע֨ת֙ לעבד֔ך ש֣ם מ֤ת אֽהרן֙ ויקב֣ר ש֔ם בכל־לבֽבך֥ ובכל־נפשך֖ ובכל־מאדֽך׃
this sequence of  5  words is too long!
sequence audio length:  30.604
sequence text length(in tokens):  89
text:  וכל־יתד֥ת הֽחצ֖ר נחֽשת׃ וידב֥ר יהו֖ה אל־מש֥ה לאמֽר׃ בכל־לבֽבך֥ ובכל־נפשך֖ למ֥ען חיֽיך׃ ויש֗בו ובאת֡
this sequence of  5  words is too long!
sequence audio length:  31.068375
sequence text length(in tokens):  101
text:  והב֤יא את֨נו֙ אל־הא֣רץ הז֔את והקרבת֥ם אש֖ה לֽיהוֽה׃ ורח֥ץ במ֖ים וטמ֥א עד־העֽרב׃ ולֽא־תש֥א את֖ה לבדֽך׃ יהוש֤ע בן־נון֙ נ֔ער
this sequence of  5  words is too long!
sequence audio length:  30.4646875
sequence text length(in tokens):  1

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [], 'begin_suppress_tokens': [220, 50257]}


this sequence of  5  words is too long!
sequence audio length:  33.0188125
sequence text length(in tokens):  65
text:  וידב֥ר יהו֖ה אל־מש֥ה לאמֽר׃ והנ֛פש וכרמך֖ ל֥א תזמֽר׃ כ֣י אח֤י אב֨יה֙ ה֔וא ויה֗י
this sequence of  5  words is too long!
sequence audio length:  30.418125
sequence text length(in tokens):  101
text:  ל֥א תפא֖ר אֽחר֑יך וא֨לה֙ בנ֣י רעוא֔ל למ֤ה הרע֨ת֙ לעבד֔ך ש֣ם מ֤ת אֽהרן֙ ויקב֣ר ש֔ם בכל־לבֽבך֥ ובכל־נפשך֖ ובכל־מאדֽך׃
this sequence of  5  words is too long!
sequence audio length:  30.604
sequence text length(in tokens):  89
text:  וכל־יתד֥ת הֽחצ֖ר נחֽשת׃ וידב֥ר יהו֖ה אל־מש֥ה לאמֽר׃ בכל־לבֽבך֥ ובכל־נפשך֖ למ֥ען חיֽיך׃ ויש֗בו ובאת֡
this sequence of  5  words is too long!
sequence audio length:  31.068375
sequence text length(in tokens):  101
text:  והב֤יא את֨נו֙ אל־הא֣רץ הז֔את והקרבת֥ם אש֖ה לֽיהוֽה׃ ורח֥ץ במ֖ים וטמ֥א עד־העֽרב׃ ולֽא־תש֥א את֖ה לבדֽך׃ יהוש֤ע בן־נון֙ נ֔ער
this sequence of  5  words is too long!
sequence audio length:  30.4646875
sequence text length(in tokens):  1

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [], 'begin_suppress_tokens': [220, 50257]}


RuntimeError: [enforce fail at inline_container.cc:595] . unexpected pos 5545777088 vs 5545776988

In [ ]:
kwargs = {
    "dataset_args": "config: he, split: test",
    "language": "he",
    "model_name": "he",
    "finetuned_from": "openai/whisper-medium",
    "tasks": "automatic-speech-recognition",
    "tags": "hf-asr-leaderboard",
}

In [ ]:
# trainer.save_model()

In [ ]:
trainer.push_to_hub(**kwargs)
# processor.push_to_hub("cantillation" +training_args.output_dir[1:])

In [ ]:
trainer.lr_scheduler.get_lr()

In [ ]:
processor.tokenizer.decode(train_data[26]["labels"])

In [ ]:
train_data.prob_for_num_of_parts = [1]

In [ ]:

from datetime import datetime


def log_training_to_markdown_file(training_args, training_loss, epoch, step, validation_loss, f1, recall, precision, filename="training_log.md"):
    # Get the current date and time
    now = datetime.now()

    # Format the date and time as a string
    date_time = now.strftime("%m/%d/%Y, %H:%M:%S")

    with open(filename, 'a') as f:
        f.write(f"| {date_time} | {training_args.output_dir } | {training_args.per_device_train_batch_size} | {training_args.gradient_accumulation_steps} | {training_args.learning_rate} | {training_args.warmup_steps} | {training_args.max_steps} | {training_args.gradient_checkpointing} | {training_args.gradient_checkpointing_kwargs} | {training_args.fp16} | {training_args.evaluation_strategy} | {training_args.per_device_eval_batch_size} | {training_args.predict_with_generate} | {training_args.generation_max_length} | {training_args.save_steps} | {training_args.eval_steps} | {training_args.logging_steps} | {training_args.report_to} | {training_args.load_best_model_at_end} | {training_args.metric_for_best_model} | {training_args.greater_is_better} | {training_args.push_to_hub} | {training_loss} | {epoch} | {step} | {validation_loss} | {f1} | {recall} | {precision} |\n")

def create_markdown_file_with_headers(filename="training_log_new.md"):
    with open(filename, 'w') as f:
        f.write("| Date Time | Repo Name | Batch Size | Gradient Accumulation Steps | Learning Rate | Warmup Steps | Max Steps | Gradient Checkpointing | Gradient Checkpointing Kwargs | FP16 | Evaluation Strategy | Eval Batch Size | Predict with Generate | Max Length | Save Steps | Eval Steps | Logging Steps | Report To | Load Best Model at End | Metric for Best Model | Greater is Better | Push to Hub | Training Loss | Epoch | Step | Validation Loss | f1 | recall | precision |\n")
        f.write("|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|----|---|---|\n")

# Create the Markdown file with headers
#create_markdown_file_with_headers()
        

def get_logs_with_step(trainer, step = 1500):
    # Initialize an empty dictionary to store the merged logs
    merged_logs_with_step = {}

    # Iterate over the log history
    for log in trainer.state.log_history:
        # Check if the 'step' key exists in the log and if it equals the provided step
        if 'step' in log and log['step'] == step:
            # If it does, merge the log into the merged_logs_with_step dictionary
            merged_logs_with_step.update(log)

    # Return the merged logs
    return merged_logs_with_step


# Get the training loss
training_loss = trainer_state.training_loss
# Get the step and epoch from the TrainerState
step = trainer.state.global_step
epoch = trainer.state.epoch

# Get the log history at the specified step
history = get_logs_with_step(trainer,training_args.max_steps)
# Get the evaluation details from the log history
validation_loss = history['eval_loss']
f1 = history['eval_f1']
recall = history['eval_recall']
precision = history['eval_precision']

# Log the training details
log_training_to_markdown_file(training_args, training_loss, epoch, step, validation_loss, f1, recall, precision, filename="training_log_new.md")


In [ ]:
#load the markdown file
from IPython.display import Markdown

# Open the file in read mode
with open('training_log_new.md', 'r') as file:
    # Read the content of the file
    content = file.read()

# Display the content as Markdown
display(Markdown(content))

| Model Name | Model Name | data | steps | lr |
|----------|----------|----------|--------|--------|
| whisper-medium-he-teamim-base | medium | all | 10,000 | 3e-5 |
| whisper-medium-he-teamim-ashkenazi-01 | base | ashkenazi | 9,000 | 1e-6 |




In [ ]:
# run a web server to see the tensorboard
!tensorboard --logdir ./whisper-medium-he-teamim-aviv-base --port 6006 --bind_all